In [ ]:
from IPython.core.display import HTML
with open('./style.css') as f:
    css = f.read()
HTML(css)

# Conclusion / Testing

In [ ]:
import chess                                       # Simulate the chess game
import chess.gaviota                               # Load Gaviota Tablebase
from IPython.display import display, clear_output  # Better visualization and display of the chess board
import pickle
import os.path

In [ ]:
%%capture
%run RetrogradeAnalysis.ipynb

## Testing

<b><mark>test_dtm_with_gaviota: $2^E$ $\times$ $\mathbb{N}$ $\times$ PIECE_STRING $\rightarrow$ $\Omega$, with $E \in$ ENDGAME</mark></b>

We have now created a tablebase, which allows us to determine the depth-to-mate (DTM) for any given board af an endgame situation. We would now like to verify our results, by comparing the calculated DTMs to the Gaviota tablebase. The test_dtm_with_gaviota function does exactly that. It receives three arguments: 
1. A set $S_n$ to be tested, as calculated using our Retrograde Analysis. 
2. The expected value of the DTM, i.e. $n$
3. The piece_string

It will then iterate through every board within $S_n$ and probe the Gaviota tablebase for its DTM value. If the Gaviota DTM and the expected DTM do not match, it will throw an Assertion error. 

In [ ]:
def test_dtm_with_gaviota(test_set, expected_dtm, piece_str):

    with chess.gaviota.open_tablebase("./gaviota") as tablebase:
        
        #Check if every set in test set is in gaviota
        for rep in test_set:
            board = int_to_board(rep, piece_str)
            gaviota_dtm = abs(tablebase.probe_dtm(board))
            if not gaviota_dtm == expected_dtm:
                    with open(f'./tests/{piece_str}', 'a') as f3:
                        f3.write(f"Assertion failed for board {board} with gaviota dtm = {gaviota_dtm} but expected = {expected_dtm}\n")
            assert gaviota_dtm == expected_dtm, f"FAILED: {board.epd()} -> expected_dtm ({expected_dtm}) != gaviota_dtm ({gaviota_dtm})"
        

<b><mark>test_dtm_zero: $(2^E)^n$ $\times$ PIECE_STRING $\rightarrow$ $\Omega$, with $E \in$ ENDGAME</mark></b>

However, we also have to account for those boards which are left in $S$ after the Retrograde Analysis. As they have not been sorted into an $S_n$, they are positions in which neither player can force a checkmate. They also receive a DTM of zero, just like the checkmate boards. The next test function is designed to test those boards. First, every $S_n$ from the list of sets, is subtracted from $S$ to get the remaining boards. Then we can iterate through each of these and assert that the Gaviota DTM must be zero for all of them. 

In [ ]:
def test_dtm_zero(list_of_sets, s, piece_str):
    with chess.gaviota.open_tablebase("./gaviota") as tablebase:
        c = 0
        for s_n in list_of_sets:
            s -= s_n
        
        for b in s:
            c += 1
            board = int_to_board(b, piece_str)
            gaviota_dtm = tablebase.probe_dtm(board)
            assert gaviota_dtm == 0, f"FAILED: {board.epd()} -> expected_dtm ({0}) != gaviota_dtm ({gaviota_dtm})"
                
        print(f"\nFound {c} non-checkmate positions with DTM zero")

<b><mark>test_all_sets: PIECE_STRING $\rightarrow$ $\Omega$</mark></b>

The final function puts it all together. It starts by generating $S$ (or loading it if available). First it will ensure that all the DTM = 0 cases are correct according to Gaviota. Lastly it iterates through each set $S_n$ and runs the test_dtm_with_gaviota function. Depending on the size of the tablebase, some tests may take several hours to complete. Therefore, after every successful pass, the test results are also written to a local file. 

In [ ]:
def test_all_sets(piece_str):
    if not os.path.exists(f"./tests/s_set_{piece_str}"):
        s = generate_s(piece_str)
        with open(f'./tests/s_set_{piece_str}', 'wb') as f:
            pickle.dump(s, f)
    else:
        with open(f'./tests/s_set_{piece_str}', 'rb') as f:
            s = pickle.load(f)
            print(f"Loading s for {piece_str}")
    print(len(s))
    
    with open(f'./tables/{piece_str}', 'rb') as f:
        list_of_sets = pickle.load(f)
        print(f"Loading list of sets for {piece_str}")
    
    print("Testing for DTM = 0")
    # Verify that all positions in s, which are not in list of sets, have Gaviota DTM = 0
    test_dtm_zero(list_of_sets, s, piece_str)
    
    # Check each set
    for i in range(0, len(list_of_sets)):
        print(f"Testing dtm values for set s_{i}...")
        
        test_dtm_with_gaviota(list_of_sets[i], i, piece_str)
        
        with open(f'./tests/{piece_str}', 'a') as f3:
            f3.write(f"s_{i} passed\n")
        print(f"s_{i} passed")

In [ ]:
test_all_sets("Kknb")